In [4]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

%matplotlib

Using matplotlib backend: MacOSX


In [5]:
def plot_loss(outputs_epoch, synth, title, k):
    plt.close()
    synth = synth.detach().numpy()
    for i, outputs in enumerate(outputs_epoch):
        if (i%k==1) and (i >20):
            outputs = np.array([output.detach().numpy()[0] for output in outputs])
            plt.plot(outputs - synth, alpha=1 - 0.5*((i+1)/len(outputs_epoch))**2, label=str(i))
    outputs = np.array([output.detach().numpy()[0] for output in outputs_epoch[-1]])
    plt.plot(outputs - synth, alpha=0.5, label="last")
    plt.legend()
    plt.title(title)

In [6]:
def plot_results(outputs_epoch, synth, title, k):
    plt.close()
    for i, outputs in enumerate(outputs_epoch):
        if i%k==1:
            outputs = np.array([output.detach().numpy()[0] for output in outputs])
            plt.plot(outputs, alpha=((i+1)/len(outputs_epoch))**2, label=str(i))
    outputs = np.array([output.detach().numpy()[0] for output in outputs_epoch[-1]])
    plt.plot(outputs, alpha=1, label="last")
    plt.plot(synth.detach().numpy(), label="true")
    plt.legend()
    plt.title(title)

In [7]:
def fun_exp(x):
    x_bar = torch.mean(x)
    return torch.exp(2*x_bar + 3*x_bar**3)

In [8]:
def fun_pol(x):
    x_bar = torch.max(x)
    return x_bar + 2*x_bar + 3*x_bar**2

In [9]:
def fun_cos(x):
    x_bar = torch.mean(x)
    return torch.cos(2*x_bar + 3*x_bar**2)

In [10]:
class Net(nn.Module):
    
    def __init__(self, N):
        super().__init__() # Runs initialisation of nn.Module
        self.fc1 = nn.Linear(1, 64)
        self.fc2 = nn.Linear(64, 128)
        self.fc3 = nn.Linear(128,512)
        self.fc4 = nn.Linear(512, 256)
        self.fc5 = nn.Linear(256, 1)
        
        self.N = N

    def phi(self, x):
        x = F.relu(self.fc1(x.view(-1)))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = F.relu(self.fc4(x))
        x = self.fc5(x)
        return x

        
    def forward(self, x):
        N = self.N
        y = torch.zeros(N, 1)
        for i in range(N):
            y[i] = self.phi(x[i].view(-1))
        y = torch.sum(y, 0) / N
        return y

In [11]:
class DeepSet(nn.Module):

    def __init__(self, N):
        super().__init__() # Runs initialisation of nn.Module
        self.fc1 = nn.Linear(1, 64)
        self.fc2 = nn.Linear(64, 64)
        self.fc3 = nn.Linear(64, 64)
        self.fc4 = nn.Linear(64, 128)
        self.fc5 = nn.Linear(128, 1024)
        
        self.psi1 = nn.Linear(1024, 512)
        self.psi2 = nn.Linear(512, 256)
        self.psi3 = nn.Linear(256, 1)
        
        self.N = N

    def phi(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = F.relu(self.fc4(x))
        x = self.fc5(x)
        return x

    def psi(self, x):
        x = F.relu(self.psi1(x))
        x = F.relu(self.psi2(x))
        x = self.psi3(x)
        return x
        
    def forward(self, x):
        N = self.N
        y = torch.zeros(N, 1024)
        for i in range(N):
            y[i] = self.phi(x[i].view(-1))
        y = torch.sum(y, 0) / N
    
        return self.psi(y)

In [12]:
class PointNet(nn.Module):

    def __init__(self, N):
        super().__init__() # Runs initialisation of nn.Module
        self.fc1 = nn.Linear(1, 64)
        self.fc2 = nn.Linear(64, 64)
        self.fc3 = nn.Linear(64, 64)
        self.fc4 = nn.Linear(64, 128)
        self.fc5 = nn.Linear(128, 1024)
        
        self.psi1 = nn.Linear(1024, 512)
        self.psi2 = nn.Linear(512, 256)
        self.psi3 = nn.Linear(256, 1)
        
        self.N = N

    def phi(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = F.relu(self.fc4(x))
        x = self.fc5(x)
        return x

    def psi(self, x):
        x = F.relu(self.psi1(x))
        x = F.relu(self.psi2(x))
        x = self.psi3(x)
        return x
        
    def forward(self, x):
        N = self.N
        y = torch.zeros(N, 1024)
        for i in range(N):
            y[i] = self.phi(x[i].view(-1))
        y = y.max(0).values
        return self.psi(y)

In [13]:
N = 10
n_data = 1000
x = torch.normal(torch.zeros(n_data, N), torch.ones(n_data, N)) 
synth = torch.zeros(n_data)
for i in range(0,n_data):
    synth[i] = fun_cos(x[i, :])

In [14]:
batches = 10
eps = 3*1e-2
net = DeepSet(N).float()
optimizer = optim.Adam(net.parameters(), lr=0.01) # Corresponds to evything that is adjustable
criterion = nn.MSELoss()
EPOCHS = 100
outputs_epoch = []
for epoch in range(EPOCHS):
    outputs = []
    net.zero_grad()
    for i in range(batches):
        k = int(n_data /batches)
        dataX = x[i:i+k,:]
        dataY = synth[i:i+k]
        tot_loss = 0
        net.zero_grad()
        for j , (X, Y) in enumerate(zip(dataX, dataY)):
            output = net(X)
            loss = criterion(output, Y)
            outputs.append(output)
            tot_loss += loss
        tot_loss.backward() # Backpropagate the loss
        optimizer.step() # Adjusts the steps
    outputs_epoch.append(outputs)
    print('LOSS = ', tot_loss)
    if tot_loss < eps:
        break


/Users/victoriadesmarquest/env/lib/python3.8/site-packages/torch/nn/modules/loss.py:528: UserWarning: Using a target size (torch.Size([])) that is different to the input size (torch.Size([1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


LOSS =  tensor(21.0915, grad_fn=<AddBackward0>)
LOSS =  tensor(23.3461, grad_fn=<AddBackward0>)
LOSS =  tensor(20.4146, grad_fn=<AddBackward0>)
LOSS =  tensor(17.4278, grad_fn=<AddBackward0>)
LOSS =  tensor(11.3501, grad_fn=<AddBackward0>)
LOSS =  tensor(8.4097, grad_fn=<AddBackward0>)
LOSS =  tensor(4.8579, grad_fn=<AddBackward0>)
LOSS =  tensor(12.2919, grad_fn=<AddBackward0>)
LOSS =  tensor(11.9594, grad_fn=<AddBackward0>)
LOSS =  tensor(9.5772, grad_fn=<AddBackward0>)
LOSS =  tensor(10.0172, grad_fn=<AddBackward0>)
LOSS =  tensor(6.8702, grad_fn=<AddBackward0>)
LOSS =  tensor(5.5485, grad_fn=<AddBackward0>)
LOSS =  tensor(3.1869, grad_fn=<AddBackward0>)
LOSS =  tensor(1.6258, grad_fn=<AddBackward0>)
LOSS =  tensor(0.8539, grad_fn=<AddBackward0>)
LOSS =  tensor(2.0254, grad_fn=<AddBackward0>)
LOSS =  tensor(6.1966, grad_fn=<AddBackward0>)
LOSS =  tensor(4.1875, grad_fn=<AddBackward0>)
LOSS =  tensor(2.3819, grad_fn=<AddBackward0>)
LOSS =  tensor(0.6669, grad_fn=<AddBackward0>)
LOSS 

In [22]:
0.0009 < 1e-3

True

In [45]:
plot_loss(outputs_epoch, synth, "DeepSet batches", 20)

In [16]:
plot_results(outputs_epoch, synth, "DeepSet 10 batches", 20)

In [53]:
net = DeepSet(N)

optimizer = optim.Adam(net.parameters(), lr=0.01) # Corresponds to evything that is adjustable
criterion = nn.MSELoss()
EPOCHS = 10
outputs_epoch = []
for epoch in range(EPOCHS):
    outputs = []
    for X, Y in zip(x, synth):
        net.zero_grad()
        output = net(X)
        outputs.append(output)
        #print(output)
        loss = criterion(output, Y.view(-1))
        loss.backward() # Backpropagate the loss
        optimizer.step() # Adjusts the steps
    print('LOSS = ', loss)
    outputs_epoch.append(outputs)

LOSS =  tensor(0.0297, grad_fn=<MseLossBackward>)
LOSS =  tensor(0.0022, grad_fn=<MseLossBackward>)
LOSS =  tensor(0.0183, grad_fn=<MseLossBackward>)
LOSS =  tensor(0.0325, grad_fn=<MseLossBackward>)
LOSS =  tensor(0.0346, grad_fn=<MseLossBackward>)
LOSS =  tensor(0.0947, grad_fn=<MseLossBackward>)
LOSS =  tensor(0.1246, grad_fn=<MseLossBackward>)
LOSS =  tensor(0.1908, grad_fn=<MseLossBackward>)
LOSS =  tensor(0.2005, grad_fn=<MseLossBackward>)
LOSS =  tensor(0.2029, grad_fn=<MseLossBackward>)


In [54]:
plot_results(outputs_epoch, synth, "DeepSet", 2)

In [12]:
net = PointNet(N)

optimizer = optim.Adam(net.parameters(), lr=0.05) # Corresponds to evything that is adjustable
criterion = nn.MSELoss()
EPOCHS = 10
outputs_epoch = []
for epoch in range(EPOCHS):
    outputs = []
    for X, Y in zip(x, synth):
        net.zero_grad()
        output = net(X)
        outputs.append(output)
        #print(output)
        loss = criterion(output, Y.view(-1))
        loss.backward() # Backpropagate the loss
        optimizer.step() # Adjusts the steps
    print('LOSS = ', loss)
    outputs_epoch.append(outputs)

LOSS =  tensor(0.4140, grad_fn=<MseLossBackward>)
LOSS =  tensor(0.2854, grad_fn=<MseLossBackward>)
LOSS =  tensor(0.0718, grad_fn=<MseLossBackward>)
LOSS =  tensor(0.1102, grad_fn=<MseLossBackward>)
LOSS =  tensor(0.3298, grad_fn=<MseLossBackward>)
LOSS =  tensor(1.2763, grad_fn=<MseLossBackward>)
LOSS =  tensor(0.2824, grad_fn=<MseLossBackward>)
LOSS =  tensor(0.0687, grad_fn=<MseLossBackward>)
LOSS =  tensor(0.0205, grad_fn=<MseLossBackward>)
LOSS =  tensor(0.4317, grad_fn=<MseLossBackward>)


In [19]:
plot_results(outputs_epoch, synth, "PointNet", 5)

In [58]:
net = Net(N)

optimizer = optim.Adam(net.parameters(), lr=0.01) # Corresponds to evything that is adjustable
criterion = nn.MSELoss()
EPOCHS = 100
outputs_epoch = []
for epoch in range(EPOCHS):
    outputs = []
    for X, Y in zip(x, synth):
        net.zero_grad()
        output = net(X)
        outputs.append(output)
        #print(output)
        loss = criterion(output, Y.view(-1))
        loss.backward() # Backpropagate the loss
        optimizer.step() # Adjusts the steps
    print('LOSS = ', loss)
    outputs_epoch.append(outputs)

LOSS =  tensor(0.0583, grad_fn=<MseLossBackward>)
LOSS =  tensor(0.0149, grad_fn=<MseLossBackward>)
LOSS =  tensor(0.0414, grad_fn=<MseLossBackward>)
LOSS =  tensor(0.0373, grad_fn=<MseLossBackward>)
LOSS =  tensor(0.0388, grad_fn=<MseLossBackward>)
LOSS =  tensor(0.0442, grad_fn=<MseLossBackward>)
LOSS =  tensor(0.0494, grad_fn=<MseLossBackward>)
LOSS =  tensor(0.0513, grad_fn=<MseLossBackward>)
LOSS =  tensor(0.0499, grad_fn=<MseLossBackward>)
LOSS =  tensor(0.0609, grad_fn=<MseLossBackward>)
LOSS =  tensor(0.0605, grad_fn=<MseLossBackward>)
LOSS =  tensor(0.0608, grad_fn=<MseLossBackward>)
LOSS =  tensor(0.0581, grad_fn=<MseLossBackward>)
LOSS =  tensor(0.0631, grad_fn=<MseLossBackward>)
LOSS =  tensor(0.0614, grad_fn=<MseLossBackward>)
LOSS =  tensor(0.0693, grad_fn=<MseLossBackward>)
LOSS =  tensor(0.0616, grad_fn=<MseLossBackward>)
LOSS =  tensor(0.0617, grad_fn=<MseLossBackward>)
LOSS =  tensor(0.0681, grad_fn=<MseLossBackward>)
LOSS =  tensor(0.0772, grad_fn=<MseLossBackward>)


In [59]:
plot_results(outputs_epoch, synth, "Net batches", 20)